---
---

### Algorithm

**Задача:** найти PDF $\rho$ в момент времени $t$ на Чебышевской сетке из уравнений
$$
    dx = f(x, t) \, dt + d\beta,
    \quad
    \frac{\partial \rho}{\partial t} = \Delta \rho - div \left[ f(x, t) \rho \right],
    \quad
    \rho(x, 0) = \rho_0(x).
$$

**Нужно** передать в крест функцию (step), которая вычисляет значения $\rho_{k+1}$ в заданном (произвольном) наборе точек $X$ Чебышевской сетки. Это позволит проинтерполировать $\rho$ на $(k+1)$-ом шаге и перейти к следующему шагу. При этом предполагается, что интерполянт для $k$-ого шага уже известен.

> На нулевом шаге имеем $\rho_0(x)$, заданную как функцию от $x$ и можем построить интерполянт очевидным образом.

> Если не используем крест (работаем в полном numpy формате), то $X$ - это полный набор точек Чебышевской сетки (алгоритм при этом остается прежним).

> Алгоритм расписан для splitting схемы первого порядка, поэтому для решения ОДЕ можем использовать метод Эйлера без потери точности.

**Работа функции step(X, I)**

> X - произвольный набор точек Чебышевской сетки (ndarray [dims, pois] of float)

> I - набор индексов узлов сетки, соответствующий точкам X (ndarray [dims, pois] of int)

**1** Найти прообразы $\widehat{X}$ (соответствуют предыдущему $k$-ому шагу) для заданного набора точек $X$ Чебышевской сетки, которые приводили бы траекторию детерминированного уравнения ($\beta = 0$) в точки $X$ на $(k+1)$-ом шаге, интегрируя назад уравнение
$$
    \frac{\partial \, x}{\partial \, t} = f(x, t),
    \quad
    x_{k+1} = X,
    \quad
    x_{k} = \widehat{X} = ?.
$$

**2** Используя известный интерполянт на $k$-ом шаге, вычислить значения PDF $\widehat{\rho}$ в точках $\widehat{X}$

**3** Решить PDE с однородными граничными условиями Дирихле для $(k+1)$-ого шага, используя дифференциальную матрицу Чебышева
$$
    \frac{\partial v}{\partial t} = \Delta v,
    \quad
    v_{k} = \widehat{\rho},
    \quad
    v_{k+1} = v = ?.
$$

**4** Решить ODE для $k+1$-ого шага
$$
    \frac{\partial \, \log{w}}{\partial \, t} = -tr \left[ f_x(x, t) \right],
    \quad
    w_{k} = v,
    \quad
    w_{k+1} = w = ?.
$$

**5** Вернуть значение $w$ как приближение $\rho(x)$ в заданном (произвольном) наборе точек $X$ Чебышевской сетки на $(k+1)$-ом шаге.

---
---

# Problems

**1** Решение PDE (этап 3) в соответствии с алгоритмом, получается, должно искаться только на части узлов сетки (соответствующей массиву запрошенных крестом точек). Также из его решения необходимо извлечь только те значения, которые соответствуют запрошенным узлам сетки $x$. Не понятно, как это можно сделать.

> Проблема актуальна только при использовании креста (в полном формате набор точек - это сразу вся сетка)

**2** Точки для $\rho$ убегают из отрезка интерполяции [-3, 3]. Что с ними лучше всего делать?

**3** Уравнение на $\log{w}$, получается, предполагает только положительные PDF, то есть, занулять как-бы нельзя (а в интерполяции вообще отрицательные значения могут вылазить, особенно, если точка попадает вне отрезка)?

**4** Пусть в исходном уравнении $f(x, t) = 0$ (чисто "диффузионный" процесс), тогда
$$
    \widehat{x} = x,
    \quad
    \widehat{\rho} = \rho_k,
    \quad
    v_{k+1} = v = e^{h} \left( e^D \otimes \ldots \otimes e^D \right) \rho_k,
$$
а для последнего ODE (этап 4) имеем
$$
    \frac{\partial \, \log{w}}{\partial \, t} = 0,
    \quad
    w_{k} = v,
$$
то есть $w_{k+1} = w = e^{v}$, и для финального решения на $(k+1)$-ом шаге получаем
$$
    \rho_{k+1}(x) = \exp{ \left[
        e^{h} \left( e^D \otimes \ldots \otimes e^D \right) \rho_k
    \right]}.
$$
Верна ли данная формула?

---
---

### Solution of 1D Ornstein–Uhlenbeck process

Ornstein–Uhlenbeck process
$$
    \frac{d \, x}{d \, t} = - \lambda x(t) + w(t),
    \quad
    x(0) = x_0,
    \quad
    \lambda > 0,
$$
or
$$
    d \, x = - \lambda x(t) d \, t + d \, \beta(t),
    \quad
    x(0) = x_0,
    \quad
    \lambda > 0,
$$

Its solution is
$$
    x(t) = e^{-\lambda t} x(0) + \int_0^t e^{-\lambda (t - \tau)} d \, \beta(\tau),
$$
and covariance matrix is
$$
    C(t) = \frac{q}{2\lambda} e^{-\lambda |t|},
$$
where $q$ is spectral density of the one dimensional white noise process $w(t)$.

The probability density function (PDF) $\rho(x, t)$ satisfies the Fokker–Planck equation
$$
   \frac{\partial \, \rho}{\partial \, t} =
       \lambda \frac{\partial}{\partial \, x} (x \rho) +
       \frac{1}{2} \frac{\partial^2 \, \rho}{\partial \, x^2},
$$
with transition probability
$$
    \rho(x, t | x_0, t_0=0) = 
        \sqrt{
            \frac{\lambda}{\pi \left( 1 - e^{-2 \lambda t} \right)}
        }
        e^{\frac
            {-\lambda \left( x - x_0 e^{-\lambda t} \right)^2}
            {1 - e^{-2 \lambda t}}
        }.
$$

In [1]:
import sys
import time

import numpy as np
from numpy import kron as kron

import scipy.sparse as sp
from scipy.linalg import expm as expm
from scipy.interpolate import RectBivariateSpline

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation, rc

from IPython.display import HTML

sys.path.append('./../lib')
from intertrain import Intertrain

In [2]:
# SDE parameters : f = A x, df / dx = A (dimension = 1)

A = -1.

def func_f(x):
    return A*x

def func_f_der(x):
    if not isinstance(x, np.ndarray):
        return A
    return A*np.ones(x.shape)

def func_r0(x):
    s = 1.
    m = 0.
    r = 1./np.sqrt(2.*np.pi * s**2)
    r*= np.exp(-1. * (x-m)**2 / 2. / s**2)
    return r.reshape(-1)
    # return np.exp(-0.5 * np.diag(x.T@x))

def func_r_trans(x, t, x0):
    l = -1. * A
    e = np.exp(-1. * l * t)
    r = np.sqrt(l / np.pi /  (1. - e*e))
    r*= np.exp(-1. * l * (x - x0 * e)**2 / (1. - e*e))
    return r

In [3]:
# Time grid

t_min = 0.
t_max = 1.
t_poi = 10 # including t_min and t_max (>= 2)

m = t_poi
h = (t_max - t_min) / (t_poi - 1)

T = np.linspace(t_min, t_max, t_poi)

In [4]:
# Spatial grid

n = 40
l = [-3., 3.]

In [5]:
def solve_ode_x(func, x0, h):
    f = func(x0)
    x = x0 + h * f
    return x

def solve_ode_r(func, x0, r0, h):
    f = -1. * np.trace(func(x0))
    
    #r = (1. - h * np.trace(f)) * r0
    r = (1. - h * f) * r0
    return r
    
    r_log = np.log(r0) + h * f
    r = np.exp(r_log)
    return r

def solve_pde_r(r0):
    return Z@r0

def step(X, I):
    print(I)
    X0 = solve_ode_x(func_f, X, -1. * h)
    r0 = IT0.calc(X0)
    u = solve_pde_r(r0)
    w = solve_ode_r(func_f_der, X, u, h)
    r = w
    return r

In [7]:
IT = Intertrain(n=[n], l=[l], with_tt=False)
IT.init(func_r0).prep()

I = np.eye(n)
D = IT.dif1()
# D = D[1:-1, 1:-1] # for BC
Z = expm(D)
Z = np.exp(h) * Z # kron(Z, Z)

for i in range(1):
    IT0 = IT.copy()
    IT.init(step, opts={'is_f_with_i': True}).prep()
print('OK')

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]]
OK


In [ ]:
# Plot distribution (initial and final) on the spatial grid

fig = plt.figure(figsize=(6, 6))
gs = mpl.gridspec.GridSpec(
    ncols=1, nrows=1, left=0.01, right=0.99, top=0.99, bottom=0.01,
    wspace=0.4, hspace=0.3, width_ratios=[1], height_ratios=[1]
)

x0 = X.reshape(-1)
#r1 = R_list[0].reshape(-1)
r2 = IT.calc(X).reshape(-1)
ax = fig.add_subplot(gs[0, 0])
#ax.plot(X.reshape(-1), r1, label='t = %-8.2e'%t_min)
ax.plot(X.reshape(-1), r2, label='t = %-8.2e'%t_max)
ax.set_title('Probability density function')
ax.set_xlabel('x')
ax.set_ylabel('r')
ax.legend(loc='best')

plt.show()